In [35]:
from functions import start_selenium, scrape_chapel, scrape_independent, scrape_rickshaw, parse_chapel_soups, parse_independent_soup, parse_rickshaw_soups, spotify_connect
from datetime import datetime
from dotenv import load_dotenv
import os
from openai import OpenAI
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from time import time

def get_soup(url, driver):
    try:
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        return BeautifulSoup(html, 'html.parser')
    except TimeoutException:
        print(f"Timeout on {url}, skipping...")
        return None


url = "https://www.bottomofthehill.com/calendar.html"



In [33]:
driver = start_selenium()
soup = get_soup(url, driver)
driver.quit()

In [63]:
# bottom of the hill is kind of tricky because they have many empty tags with the same date_selector
# and many artists play in the same event

def parse_bottom_of_the_hill(soup):
    artist_selector = 'big[class="band"]'
    date_selector = 'span[class="date"]'
    event_selector = 'td[style*="background-color: rgb(204, 204, 51)"]'
    genre_selector = 'span[class="genre"]'

    event_containers = soup.select(event_selector)
    events = []

    for container in event_containers:
        artist_elems = container.select(artist_selector)
        artists = [artist_elem.text.strip() for artist_elem in artist_elems if artist_elem and artist_elem.text.strip()]
        date_elems = container.select(date_selector)
        dates = [date_elem.text.strip() for date_elem in date_elems if date_elem and date_elem.text.strip()]
        artists = [artist.replace('\n', ' ').strip() for artist in artists]
        date = ''.join([date.replace('\n', ' ').strip() for date in dates])
        genre_elems = container.select(genre_selector)
        genres = [genre_elem.text.strip() for genre_elem in genre_elems if genre_elem and genre_elem.text.strip()]
        events.append({'artists':artists, 'date':date, 'genres':genres})
    events
    for event in events:
        
parse_bottom_of_the_hill(soup)

    # # Only add if we got at minimum an artist and date
    # if artist and date_text and (artist not in set([event['artist'] for event in events])):
    #     events.append({
    #         'venue_id': venue_id,
    #         'artist': artist,
    #         'date_text': date_text,
    #         'genre': genre,
    #         'is_cancelled': is_cancelled
    #     })

[{'artists': ['Sonic Blossoms', 'Frankie & the Pool Boys', 'The Ogres'],
  'date': 'Friday September 26 2025',
  'genres': ['psychedelic space rock',
   'surf rock garage psych',
   "garage rock 'n' roll",
   'spinning the classics']},
 {'artists': ['Modern Monsters', 'The Devil in California', 'Velvet Grave'],
  'date': 'Saturday September 27 2025',
  'genres': ['heavy rock', 'heavy rock', 'rock & roll grunge']},
 {'artists': ['Cool Company', 'Nic Hanson', 'J.Lately'],
  'date': 'Sunday September 28 2025',
  'genres': ['R&B, Funk, Soul, Pop & Hip Hop',
   'R&B / Soul, Hip Hop',
   'Hip hop, R&B']},
 {'artists': ['IVRI', 'Computer Kill'],
  'date': 'Tuesday September 30 2025',
  'genres': ['R&B Soul \\ Lofi Pop', 'darkwave electronica indie pop']},
 {'artists': ['Country Risqué', 'Rose Vaughn', 'Josiah Flores'],
  'date': 'Wednesday October 1 2025',
  'genres': ['alt-country\xa0 honky tonk', 'alt-country', 'folk country']},
 {'artists': ['Emo Night Brooklyn'],
  'date': 'Thursday Octob